<a href="https://colab.research.google.com/github/Fabien-GELUS/substra-notebooks/blob/master/Mnist_DP_Substra_Example_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, you can test the example's assets and potentially run ML tasks on a public distant VM.
On Google Colab, you can see and modify the assets with the "Files" button on the left.

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

TensorFlow 1.x selected.


# Mnist with Differential Privacy

*This example is a Substra implementation of on the [Classification_Privacy tutorial](https://github.com/tensorflow/privacy/blob/master/tutorials/Classification_Privacy.ipynb) from [Tensorflow_Privacy](https://github.com/tensorflow/privacy). The structure of this example is inspired from [Substra's Titanic Example](https://github.com/SubstraFoundation/substra/blob/master/examples/titanic/)*

> [Differential privacy](https://en.wikipedia.org/wiki/Differential_privacy) (DP) is a framework for measuring the privacy guarantees provided by an algorithm. Through the lens of differential privacy, we can design machine learning algorithms that responsibly train models on private data. Learning with differential privacy provides provable guarantees of privacy, mitigating the risk of exposing sensitive training data in machine learning. Intuitively, a model trained with differential privacy should not be affected by any single training example, or small set of training examples, in its data set. This mitigates the risk of exposing sensitive training data in ML.
>  
> The basic idea of this approach, called differentially private stochastic gradient descent (DP-SGD), is to modify the gradients used in stochastic gradient descent (SGD), which lies at the core of almost all deep learning algorithms. Models trained with DP-SGD provide provable differential privacy guarantees for their input data. There are two modifications made to the vanilla SGD algorithm:
>  
> 1. First, the sensitivity of each gradient needs to be bounded. In other words, we need to limit how much each individual training point sampled in a minibatch can influence gradient computations and the resulting updates applied to model parameters. This can be done by *clipping* each gradient computed on each training point.
> 2. *Random noise* is sampled and added to the clipped gradients to make it statistically impossible to know whether or not a particular data point was included in the training dataset by comparing the updates SGD applies when it operates with or without this particular data point in the training dataset.
>  
> This tutorial uses [tf.keras](https://www.tensorflow.org/guide/keras) to train a convolutional neural network (CNN) to recognize handwritten digits with the DP-SGD optimizer provided by the TensorFlow Privacy library. TensorFlow Privacy provides code that wraps an existing TensorFlow optimizer to create a variant that implements DP-SGD.  
> &mdash; [The TensorFlow Authors][1]

[1]: https://github.com/tensorflow/privacy/blob/master/tutorials/Classification_Privacy.ipynb



## Prerequisites

In order to run this example, you'll need to:

* use Python 3
* have [Docker](https://www.docker.com/) installed
* [install the `substra` cli](https://github.com/SubstraFoundation/substra#install) (supported version: 0.6.0)

In [ ]:
! pip3 install substra==0.6.0


* [install the `substratools` library](https://github.com/substrafoundation/substra-tools) (supported version: 0.6.0)
* [pull the `substra-tools` docker images](https://github.com/substrafoundation/substra-tools#pull-from-private-docker-registry)
* have access to a Substra installation ([configure your host to a public node ip](https://doc.substra.ai/getting_started/installation/local_install_skaffold.html#network) or [install Substra on your machine](https://doc.substra.ai/getting_started/installation/local_install_skaffold.html))

In [12]:
#replace this ip by the ip of a distant VM running a substra node
public_node_ip = "127.0.0.1"

In [ ]:
! echo "{public_node_ip} substra-backend.node-1.com substra-frontend.node-1.com substra-backend.node-2.com substra-frontend.node-2.com" | sudo tee -a /etc/hosts
# Check if it's ok
! curl substra-backend.node-1.com/readiness


* create a substra profile to define the substra network to target, for instance:


In [ ]:
! substra config --profile node-1 http://substra-backend.node-1.com
! substra login --profile node-1 --username node-1 --password 'p@$swr0d44'

* checkout this repository

In [6]:
%cd /content
! git clone https://github.com/Fabien-GELUS/substra-examples/
%cd substra-examples
! git checkout mnist-dp
%cd /content/substra-examples/mnist-dp

/content
Cloning into 'substra-examples'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 365 (delta 38), reused 61 (delta 18), pack-reused 271
Receiving objects: 100% (365/365), 170.35 MiB | 38.95 MiB/s, done.
Resolving deltas: 100% (180/180), done.
/content/substra-examples
Branch 'mnist-dp' set up to track remote branch 'mnist-dp' from 'origin'.
Switched to a new branch 'mnist-dp'
/content/substra-examples/mnist-dp



All commands in this example are run from the `mnist-dp` folder.


## Data preparation

The first step will be to generate train and test data from keras.datasets.mnist

To generate the data, run:



In [7]:
%cd /content/substra-examples/mnist-dp
#! pip install --upgrade pip
#! pip install -r scripts/requirements.txt
# requirements are already satisfied in Colab, except for substratools
! pip install substratools==0.6.0

/content/substra-examples/mnist-dp


In [8]:
%cd /content/substra-examples/mnist-dp
! python scripts/generate_data.py

/content/substra-examples/mnist-dp
Loading data from keras.datasets.mnist ...
11493376/11490434 [==============================] - 0s 0us/step
Data will be generated in :  /content/substra-examples/mnist-dp/assets
File created :  train_data/features/x_train.npy
File created :  train_data/labels/y_train.npy
File created :  test_data/features/x_test.npy
File created :  test_data/labels/y_test.npy




This will create two sub-folders in the `assets` folder:

* `train_data` contains train data features and labels as numpy array files
* `test_data` contains test data features and labels as numpy array files

## Writing the objective and data manager

Both objective and data manager will need a proper markdown description, you can check them out in their respective
folders. Notice that the data manager's description includes a formal description of the data structure.

Notice also that the `metrics.py` and `opener.py` module both rely on classes imported from the `substratools` module.
These classes provide a simple yet rigid structure that will make algorithms pretty easy to write.

## Writing a simple algorithm

You'll find under `assets/algo_cnn_dp` an implementation of the cnn model in the [Classification_Privacy tutorial](https://github.com/tensorflow/privacy/blob/master/tutorials/Classification_Privacy.ipynb). Like the metrics and opener scripts, it relies on a
class imported from `substratools` that greatly simplifies the writing process. You'll notice that it handles not only
the train and predict tasks but also a lot of data preprocessing.

This algorithm measure the differential privacy guarantee after training the model:
You will see in the console the value Epsilon (ϵ) - This is the privacy budget. It measures the strength of the privacy guarantee by bounding how much the probability of a particular model output can vary by including (or excluding) a single training point. A smaller value for ϵ implies a better privacy guarantee. However, the ϵ value is only an upper bound and a large value could still mean good privacy in practice.

This value depends on:

1. The total number of points in the training data, `n`.
2. The `batch_size`.
3. The `noise_multiplier`.
4. The number of `epochs` of training.

You can find a description of the hyperparameters [here](https://github.com/tensorflow/privacy/tree/master/tutorials#parameters).



## Testing our assets

### Using asset command line interfaces

You can first test each assets with the `substratools` CLI, by running specific ML tasks in your local Python environment.



#### Training task





In [21]:
# train your model with the train_data
! python assets/algo_cnn_dp/algo.py train \
  --debug \
  --opener-path assets/dataset/opener.py \
  --data-samples-path assets/train_data \
  --output-model-path assets/model/model \
  --log-path assets/logs/train.log


substratools.utils - Module 'opener' loaded from path 'assets/dataset/opener.py'
substratools.opener - loading X from '['assets/train_data/labels', 'assets/train_data/features']'
Finding features file...
Loading features...
substratools.opener - loading y from '['assets/train_data/labels', 'assets/train_data/features']'
Finding label file...
Loading labels...
substratools.algo - launching training task
X shape: (60000, 28, 28, 1)
Train on 60000 samples
2020-08-14 15:11:18.982033: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:11:18.997134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:11:18.998074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz

In [12]:
#predict the labels of train_data with your previously trained model
! python assets/algo_cnn_dp/algo.py predict \
  --debug \
  --opener-path assets/dataset/opener.py \
  --data-samples-path assets/train_data \
  --output-predictions-path assets/pred-train.npy \
  --models-path assets/model/ \
  --log-path assets/logs/train_predict.log \
  model

substratools.utils - Module 'opener' loaded from path 'assets/dataset/opener.py'
substratools.opener - loading X from '['assets/train_data/labels', 'assets/train_data/features']'
Finding features file...
Loading features...
substratools.algo - loading model from 'assets/model/model'
2020-08-14 14:14:43.570141: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-14 14:14:43.583001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 14:14:43.583540: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-08-14 14:14:43.583799: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so

In [13]:
#calculate the score of your model on train_data predictions
! python assets/objective/metrics.py \
  --debug \
  --opener-path assets/dataset/opener.py \
  --data-samples-path assets/train_data \
  --input-predictions-path assets/pred-train.npy \
  --output-perf-path assets/perf-train.json \
  --log-path assets/logs/train_metrics.log

substratools.opener - loading y from '['assets/train_data/labels', 'assets/train_data/features']'
Finding label file...
Loading labels...
substratools.opener - loading predictions from 'assets/pred-train.npy'
substratools.metrics - launching scoring task
substratools.metrics - score: 0.84665
substratools.metrics - saving score to 'assets/perf-train.json'


#### Testing task

In [14]:
#predict the labels of test_data with your previously trained model
! python assets/algo_cnn_dp/algo.py predict \
  --debug \
  --opener-path assets/dataset/opener.py \
  --data-samples-path assets/test_data \
  --output-predictions-path assets/pred-test.npy \
  --models-path assets/model/ \
  --log-path assets/logs/test_predict.log \
  model

substratools.utils - Module 'opener' loaded from path 'assets/dataset/opener.py'
substratools.opener - loading X from '['assets/test_data/labels', 'assets/test_data/features']'
Finding features file...
Loading features...
substratools.algo - loading model from 'assets/model/model'
2020-08-14 14:14:55.584641: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-14 14:14:55.597122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 14:14:55.597900: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-08-14 14:14:55.598144: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.1

In [15]:
#calculate the score of your model on test_data predictions
! python assets/objective/metrics.py \
  --debug \
  --opener-path assets/dataset/opener.py \
  --data-samples-path assets/test_data \
  --input-predictions-path assets/pred-test.npy \
  --output-perf-path assets/perf-test.json \
  --log-path assets/logs/test_metrics.log

substratools.opener - loading y from '['assets/test_data/labels', 'assets/test_data/features']'
Finding label file...
Loading labels...
substratools.opener - loading predictions from 'assets/pred-test.npy'
substratools.metrics - launching scoring task
substratools.metrics - score: 0.8553
substratools.metrics - saving score to 'assets/perf-test.json'


### Using substra cli

Before pushing our assets to the platform, we need to make sure they work well. To do so, we can run them locally in a
Docker container. This way, if the training fails, we can access the logs and debug our code.

To test the assets, we'll use `substra run-local`, passing it paths to our algorithm of course, but also the opener,
the metrics and to the data samples we want to use. It will launch a training task on the train data, a prediction task on the test data and return the accuracy score.


In [ ]:
#you will need Docker to run this (not available in Colab)
! substra run-local assets/algo_cnn_dp \
  --train-opener=assets/dataset/opener.py \
  --test-opener=assets/dataset/opener.py \
  --metrics=assets/objective/ \
  --train-data-samples=assets/train_data \
  --test-data-samples=assets/test_data

At the end of this step, you'll find in the newly created `sandbox/model` folder a `model` file that contains your
trained model. There is also a `sandbox/pred_train` folder that contains both the predictions made by the model on
train data and the associated performance.


#### Debugging

It's more than probable that your code won't run perfectly the first time. Since runs happen in dockers, you can't
debug using prints. Instead, you should use the `logging` module from python. All logs can then be consulted at the end
of the run in  `sandbox/model/log_model.log`.

## Adding the assets to substra

### Adding the objective, dataset and data samples to substra

A script has been written that adds objective, data manager and data samples to substra. It uses the `substra` python
sdk to perform actions. It's main goal is to create assets, get their keys and use these keys in the creation of other
assets.

To run it:


In [15]:
! python scripts/add_dataset_objective.py

Adding dataset...
Adding train data samples...
100% 2/2 [00:03<00:00,  1.87s/it]
Adding test data samples...
100% 2/2 [00:01<00:00,  1.26it/s]
Associating data samples with dataset...
Adding objective...
Assets keys have been saved to /content/substra-examples/mnist-dp/assets_keys.json


This script just generated an `assets_keys.json` file in the `mnist` folder. This file contains the keys of all assets
we've just created and organizes the keys of the train data samples in folds. This file will be used as input when
adding an algorithm so that we can automatically launch all training and testing tasks.


### Adding the algorithm and training it

The script `add_train_algo_cnn.py` pushes our simple algo to substra and then uses the `assets_keys.json` file
we just generated to train it against the dataset and objective we previously set up. It will then update the
`assets_keys.json` file with the newly created assets keys (algo, traintuple and testtuple)

To run it:

In [ ]:
! python scripts/add_train_algo_cnn_dp.py

It will end by providing a couple of commands you can use to track the progress of the train and test tuples as well
as the associated scores. Alternatively, you can browse the frontend to look up progress and scores.